In [1]:
#LSTM caso 1: UNIVARIADO - CANTIDAD_VENTAS
import pyodbc
import pandas as pd
from datetime import date, datetime, timedelta
import numpy as np
from dateutil.relativedelta import relativedelta
import matplotlib.pylab as plt
%matplotlib inline
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, LSTM
from keras.optimizers import SGD, RMSprop
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest

plt.rcParams['figure.figsize' ] = (16, 9)
plt.style.use('fast')

In [4]:
sql_server = """
DRIVER={ODBC Driver 17 for SQL Server};
server=US3RN4M3;
database=AdventureWorksDW2022;
Trusted_connection=yes;
"""

In [7]:
query = """
SELECT
    CAST(OrderDate AS DATE) AS Date
    ,YEAR([OrderDate]) AS [YEAR]
    ,MONTH([OrderDate]) AS [MONTH]
    ,DAY([OrderDate]) AS [Day]
    ,DATEPART(weekday, [OrderDate]) - 1 AS [DayOfWeek]
    ,CASE
        WHEN DATEPART(weekday, [OrderDate]) IN (7, 1) THEN 1
        ELSE 0
    END AS IsWeekend
    ,[ProductKey]
    ,Sum([OrderQuantity]) AS OrderQuantity
    ,Sum([UnitPrice]) AS UnitPrice
    ,Sum([UnitPriceDiscountPct]) As UnitPriceDiscountPct
    ,Sum([DiscountAmount]) AS DiscountAmount
    ,Sum([ProductStandardCost]) AS ProductStandardCost
    ,Sum([SalesAmount]) AS SalesAmount
    FROM [AdventureWorksDW2022].[dbo].[FactInternetSales]
    group by YEAR([OrderDate]),MONTH([OrderDate]),DAY([OrderDate]),DATENAME(weekday,[OrderDate]),[OrderDate],[ProductKey]
    order by YEAR([OrderDate]) ASC,MONTH([OrderDate]) ASC,DAY([OrderDate]) ASC
"""

In [9]:
query2 = """
SELECT CAST(f.OrderDate AS DATE) AS sales_date, CAST(SUM(f.SalesAmount) AS INT) AS TotalVentas FROM FactInternetSales AS f GROUP BY CAST(f.OrderDate AS DATE) ORDER BY CAST(f.OrderDate AS DATE) ASC;
"""

In [10]:
#CONSTANTES

PASOS = 31
TRAINING_PERCENTAGE = 0.8
N_PREDICTIONS = 31
EPOCHS = 100
NEURONS = 31 #Mismo que el de pasos


#NOTAS DE ESTA PARTE: el número de pasos no es correlativo al No. de predicciones
#ya que, las predicciones pudieran ser de 61 días y el No. de pasos siempre serán 31, ya que bajo
#ese proceso se hizo el entrenamiento

In [11]:
#Opcion 1: Permitir que pueda ser modificada la sentencia, y mantener el try para evitar de que
# el programa se rompa
def get_sqlconnection(config_sqlServer):
    status = "inicializando...."
    try: 
        connection = pyodbc.connect(sql_server)
        status = "Conexion establecida satisfactoriamente"
    except Exception as e:
        status = "Error al establecer la conexión:"+e
    print(status)
    return connection

In [12]:
def set_index_datetime(data):
        if str(type(data) == "<class 'pandas.core.frame.DataFrame'>"):
            # data.sort_values('fecha', inplace=True)
            for column in data.columns: 
                try: 
                    pd.to_datetime(data[column])
                    data.set_index(column,inplace=True)
                    return data
                except Exception as e:  
                    pass
        else: 
            return 0

In [13]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
def create_x_y_train(data):
    values = data.values
    values = values.astype('float32')
    scaler = MinMaxScaler(feature_range=(-1, 1))
    values= values.reshape(-1, 1)
    scaled = scaler.fit_transform(values)
    reframed = series_to_supervised(scaled, PASOS, 1)
    values = reframed.values
    
    #debemos obtener el total de datos de entrenamiento
    n_train_days = int(len(values) * TRAINING_PERCENTAGE)
    print("Total de datos: ",int(len(values)))

    #80% - entrenamiento 
    train = values[:n_train_days, :]
    print("entrenamiento: ",len(train))

    #20% - prueba
    test = values[n_train_days:, :]
    print("testing: ",len(test))
    x_train, y_train = train[:, :- 1], train[:, -1]
    x_val, y_val = test[:, :- 1], test[:, -1]
    x_train = x_train.reshape((x_train.shape[0], PASOS, 1))
    x_val = x_val.reshape((x_val.shape[0], PASOS, 1))
    print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)
    return x_train, y_train, x_val, y_val, scaler, values

In [15]:
def train_model(x_train, y_train, x_val, y_val, scaler, values, data, model):
    history = model.fit(x_train, y_train, epochs=EPOCHS, batch_size=32, validation_data=(x_val, y_val), verbose=2, shuffle=False)

    #Obtener el 20% de la data
    ultimosDias = data[data.index[int(len(data)*TRAINING_PERCENTAGE)]:]
    values = ultimosDias.values
    values = values.astype('float32')
    values = values.reshape(-1, 1)
    scaled = scaler.fit_transform(values)
    reframed = series_to_supervised(scaled, PASOS, 1)
    reframed.drop(reframed.columns[[PASOS]], axis=1, inplace=True)
    values = reframed.values
    print("Meses registrados: ",len(values))
    x_test = values[len(values)-1:, :]
    print("La cantidad de días son: ",x_test.size)
    print("Dimensiones de x_test antes del reshape:", x_test.shape)
    #Tensor
    x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
    return model, x_test

In [ ]:
def crear_modeloLSTM(n_features):
    input_shape = (PASOS,n_features)
    model = Sequential()
    model.add(LSTM(NEURONS, input_shape=input_shape, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.add(Activation('linear'))
    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mse'])
    return model

In [17]:
def eliminar_anomalias(dtaframe):
    dataFrame_anomalias = dtaframe.copy()
    modeloIsolation = IsolationForest(contamination=0.05)
    modeloIsolation.fit(dataFrame_anomalias)
    anomalias = modeloIsolation.predict(dataFrame_anomalias)
    dtaframe['anomalias' ] = anomalias
    dataFrameSinAnomalias = dtaframe[dtaframe['anomalias' ] != -1]
    dataFrameSinAnomalias = dataFrameSinAnomalias.drop('anomalias', axis=1)
    return dataFrameSinAnomalias

In [18]:
def addNewValue(x_test, nuevoValor):
    x_test[0, :-1, 0] = x_test[0, 1:, 0]
    x_test[0, -1, 0] = nuevoValor
    return x_test

In [19]:
#core
with get_sqlconnection(sql_server) as cursor: 
    prepData = pd.read_sql_query(query,cursor)
    historyData = pd.read_sql_query(query2, cursor)

    

Conexion establecida satisfactoriamente


C:\Users\JoCel\AppData\Local\Temp\ipykernel_4704\1245306699.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  prepData = pd.read_sql_query(query,cursor)
C:\Users\JoCel\AppData\Local\Temp\ipykernel_4704\1245306699.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  historyData = pd.read_sql_query(query2, cursor)


In [20]:
    prepData

,Date,YEAR,MONTH,Day,DayOfWeek,IsWeekend,ProductKey,OrderQuantity,UnitPrice,UnitPriceDiscountPct,DiscountAmount,ProductStandardCost,SalesAmount
0,2010-12-29,2010,12,29,3,0,346,3,10199.9700,0.0,0.0,5736.4632,10199.9700
1,2010-12-29,2010,12,29,3,0,310,1,3578.2700,0.0,0.0,2171.2942,3578.2700
2,2010-12-29,2010,12,29,3,0,336,1,699.0982,0.0,0.0,413.1463,699.0982
3,2010-12-30,2010,12,30,4,0,310,1,3578.2700,0.0,0.0,2171.2942,3578.2700
4,2010-12-30,2010,12,30,4,0,351,1,3374.9900,0.0,0.0,1898.0944,3374.9900
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23792,2014-01-28,2014,1,28,2,0,478,3,29.9700,0.0,0.0,11.2089,29.9700
23793,2014-01-28,2014,1,28,2,0,538,5,107.4500,0.0,0.0,40.1865,107.4500
23794,2014-01-28,2014,1,28,2,0,481,1,8.9900,0.0,0.0,3.3623,8.9900
23795,2014-01-28,2014,1,28,2,0,536,2,59.9800,0.0,0.0,22.4326,59.9800


In [21]:
    prepData = set_index_datetime(prepData)
    historyData = set_index_datetime(historyData)

In [32]:
    #Switch the format on days or months
    #nota, acualmente los datos son filtrados con respecto a solo si son en  formato de día o de mes,
    #en LSTM, que incluye  datos en horarios, se tendría que modificar esta parte
    first_day = prepData.index.min() + timedelta(days=1)
    last_day = prepData.index.max() + timedelta(days=1)
    future_days = [last_day + timedelta(days=i) for i in range(N_PREDICTIONS)]
    for i in range(len(future_days)):
        future_days[i] = str(future_days[i])[:10]

In [ ]:
    #De momento solamente será la fecha
    future_data = pd.DataFrame(future_days, columns=['fecha'])

    #obtenemos el total de features con las que se hará el entrenamiento del modelo
    n_features = int(len(prepData.columns))
    prepData.columns

Index(['YEAR', 'MONTH', 'Day', 'DayOfWeek', 'IsWeekend', 'ProductKey',
       'OrderQuantity', 'UnitPrice', 'UnitPriceDiscountPct', 'DiscountAmount',
       'ProductStandardCost', 'SalesAmount'],
      dtype='object')

In [ ]:

    #creamos un modelo
    model = crear_modeloLSTM()
    for column in prepData.columns:
        # data = prepData.filter([column])
        data = prepData.filter([column])
        data.set_index(prepData.index, inplace=True)
        data = eliminar_anomalias(data)
        x_train, y_train, x_val, y_val, scaler, values = create_x_y_train(data)
        model, x_test = train_model(x_train, y_train,x_val, y_val,scaler, values, data, model)
        copy_test = x_test

c:\GitHub\pronostico\pronostico\envprediccion\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Total de datos:  23003
entrenamiento:  18402
testing:  4601
(18402, 31, 1) (18402,) (4601, 31, 1) (4601,)
Epoch 1/100


KeyboardInterrupt: 

In [ ]:
        results = []
        for i in range(N_PREDICTIONS):
            parcial = model.predict(x_test)
            results.append(parcial[0])
            x_test = addNewValue(x_test,parcial[0])
        adimen = np.array(results).reshape(-1, 1)
        inverted = scaler.inverse_transform(adimen)
        future_data[column] = inverted.astype(int)

In [ ]:
    future_data = set_index_datetime(future_data)
    prepData = historyData
    prepData.index = pd.to_datetime(historyData.index)
    future_data.index = pd.to_datetime(future_data.index)
    plt.plot(prepData.index, prepData['TotalVentas'],label='Historial {p0} - {p1}'.format(p0=str(first_day.year), p1= str(last_day.year-1)))
    plt.plot(future_data.index, future_data['TotalVentas'],label='Predicción {p0}'.format(p0=str(last_day.year)))
    xtics = data.index.union(future_data.index)[::10]

    plt.xlabel('Fecha')
    plt.ylabel('Cantidad de productos vendidos')
    plt.title('Predicción de la demanda global por meses para el {p0}'.format(p0=str(last_day.year)))



    plt.legend()
    plt.show()